In [ ]:
# # MOUNTING GOOGLE DRIVE
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# print(os.getcwd())

# wd = '/content/drive/MyDrive/CS 685/cs685_project/notebooks'
# print(os.listdir(wd))
# os.chdir(wd)
# print(os.getcwd())

In [1]:
# !pip install tokenizers
# !pip install transformers
# !pip install sentencepiece
import os
import json
import numpy as np
from tqdm import trange

from load_dataset import Text2SQLDataset
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
from tokenizers import AddedToken
from torch.utils.tensorboard import SummaryWriter
from transformers import T5TokenizerFast, T5ForConditionalGeneration
from transformers import BertModel, T5ForConditionalGeneration, T5Tokenizer, BertTokenizer

from graph import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

C:\Users\Atharva\AppData\Local\Programs\Python\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
C:\Users\Atharva\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\models\t5\tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this war

device(type='cpu')

In [2]:
# FOR PRINTING INTERMEDIATE TORCH SIZES
DEBUG_FLAG = True

In [7]:
# Define model

# class GNN(torch.nn.Module):
#     # single layer!!
#     def __init__(self, in_feats, hidden_feats, out_feats):
#         super().__init__()
#         self.conv1 = dgl.nn.GraphConv(in_feats, hidden_feats)
#         self.conv2 = dgl.nn.GraphConv(hidden_feats, out_feats)
#
#     def forward(self, g, h):
#         h = torch.relu(self.conv1(g, h))
#         h = self.conv2(g, h)
#         return h


class EncoderDecoder(nn.Module):
    def __init__(self, bert_hidden_size, t5_hidden_size, max_input_length,
                 max_output_length, bert_model, t5_model, batch_size):
        super(EncoderDecoder, self).__init__()

        self.max_input_length = max_input_length
        self.max_output_length = max_output_length
        self.t5_hidden_size = t5_hidden_size
        self.batch_size = batch_size

        self.bert = BertModel.from_pretrained(bert_model)
        self.t5 = T5ForConditionalGeneration.from_pretrained(t5_model)
#         self.linear = nn.Linear(bert_hidden_size, t5_hidden_size)
        self.linear = nn.Linear(bert_hidden_size, max_input_length*t5_hidden_size)

        self.gnn = GNNModel(max_input_length, t5_hidden_size, max_input_length)


    def forward(self, input_ids, input_mask,
                decoder_input_ids, decoder_attention_mask):

        # Encode input with BERT
        _, bert_output = self.bert(input_ids=input_ids, attention_mask=input_mask, return_dict=False)

        if DEBUG_FLAG: print(f"bert_output - {bert_output.size()}")

        # Transform BERT output to T5 input shape
        t5_input = self.linear(bert_output)
        graph_g = create_graph(t5_input)
        if DEBUG_FLAG: print(f"bert_output linear - {t5_input.size()}")

#         t5_input = t5_input.unsqueeze(1).repeat(1, self.max_output_length, 1)
#         if DEBUG_FLAG: print(f"bert_output linear unsqueeze - {t5_input.size()}")

        t5_input = t5_input.view(self.batch_size, self.max_input_length, self.t5_hidden_size)
        t5_input = t5_input.unsqueeze(0)
        if DEBUG_FLAG: print(f"t5_input - {t5_input.size()}")

#         t5_outputs = self.t5(decoder_input_ids=decoder_input_ids,
#                              decoder_attention_mask=decoder_attention_mask,
#                              encoder_outputs=t5_input
#                             )
#         if DEBUG_FLAG: print(f"t5_input logits - {(t5_outputs.logits).size()}")
#         return t5_outputs.logits

        t5_outputs = self.t5(labels=decoder_input_ids,
                             decoder_attention_mask=decoder_attention_mask,
                             encoder_outputs=t5_input,
                             return_dict = True
                            )
        # add gnn embeddigns to t5_output
        gnn_enc = self.gnn(graph_g,t5_outputs) + t5_outputs

        # give gnn + t5 encodings (Graphix layers) to final decoder

        final_outputs = self.t5(labels=decoder_input_ids,
                             decoder_attention_mask=decoder_attention_mask,
                             encoder_outputs=gnn_enc,
                             return_dict = True
                            )

        if DEBUG_FLAG: print(f"t5_input - {type(final_outputs)}")
        return gnn_enc

    def predict(self, input_ids, input_mask, batch_size, t5_tokenizer):

        _, bert_output = self.bert(input_ids=input_ids, attention_mask=input_mask, return_dict=False)
        if DEBUG_FLAG: print(f"bert_output - {bert_output.size()}")

        # Transform BERT output to T5 input shape
        t5_input = self.linear(bert_output)
        if DEBUG_FLAG: print(f"t5_input - {t5_input.size()} - {t5_input}")
        t5_input = t5_input.view(batch_size, self.max_input_length, self.t5_hidden_size)
        t5_input = t5_input.unsqueeze(0)
        if DEBUG_FLAG: print(f"t5_input - {t5_input.size()} - {t5_input}")

        t5_input = t5_input.to(torch.LongTensor) #int64)

#         t5_outputs = self.t5.generate(t5_input, max_length = 127)

        #######################################

#         # Generate initial input for T5 decoder
        start_token = t5_tokenizer.pad_token_id

#         decoder_input_ids = torch.tensor([start_token] * batch_size).unsqueeze(0)
#         decoder_attention_mask = torch.tensor([1] * batch_size).unsqueeze(0)

        decoder_input_ids = torch.tensor([start_token]).unsqueeze(0)
        decoder_attention_mask = torch.tensor([1]).unsqueeze(0)

#         decoder_input_ids = decoder_input_ids.view(decoder_input_ids.shape[1],
#                                                    decoder_input_ids.shape[0])
#         decoder_attention_mask = decoder_attention_mask.view(decoder_attention_mask.shape[1],
#                                                              decoder_attention_mask.shape[0])

        print(f"decoder_input_ids - {decoder_input_ids.size()}")
        print(f"decoder_attention_mask - {decoder_attention_mask.size()}")

        print(f"initial decoder_input_ids - {decoder_input_ids}")
        # Use the model to get output logits
        # Predict the output
        with torch.no_grad():
            for i in range(50):  # Maximum length of generated sequence
                t5_outputs = self.t5(decoder_input_ids=decoder_input_ids,
                                     decoder_attention_mask=decoder_attention_mask,
                                     encoder_outputs=t5_input)
#                 print(f"t5_outputs - {t5_outputs}")
                print(f"t5_outputs logits - {(t5_outputs.logits).size()}")

                next_token_logits = t5_outputs.logits[:, -1, :]
                print(f"next_token_logits - {next_token_logits.size()}")

#                 next_token_id = torch.argmax(next_token_logits, dim=-1)
                next_token_id = next_token_logits.argmax(1)
#                 print(f"next_token_id - {next_token_id.size()}")
#                 print(f"next_token_id.unsqueeze(-1) - {next_token_id.unsqueeze(-1).size()}")
                decoder_input_ids = torch.cat([decoder_input_ids, next_token_id.unsqueeze(-1)], dim=-1)
                decoder_attention_mask = torch.cat([decoder_attention_mask,
                                                    torch.ones_like(next_token_id.unsqueeze(-1))], dim=-1)

                if next_token_id == t5_tokenizer.eos_token_id:
                    break

                print(f"pred decoder_input_ids - {decoder_input_ids}")

#                 break

        # generated_text
        t5_outputs = t5_tokenizer.decode(decoder_input_ids.squeeze(), skip_special_tokens=True)
        #######################################

        return t5_outputs

In [8]:
# must remain same!!
def train(train_filepath, batch_size, bert_hidden_size, t5_hidden_size, lr, num_epochs,
         max_input_length, max_output_length, bert_model, t5_model):

    sub_folder_name = f"BERT_T5_lr{lr}_bs{batch_size}_{bert_model}_{t5_model}"
    models_directory = f"models/{sub_folder_name}"

    if not os.path.isdir(models_directory):
        os.makedirs(models_directory)

    # TENSORBOARD
    writer = SummaryWriter(f'tb/loss_plot/{sub_folder_name}')

    train_dataset = Text2SQLDataset(
            dir_ = train_filepath,
            mode = "train")

    train_dataloder = DataLoader(
            train_dataset,
            batch_size = batch_size,
            shuffle = True,
            collate_fn = lambda x: x,
            drop_last = True
        )

    print(f"Number of batches - {len(train_dataloder)}")

    # Define BERT and T5 tokenizers
    bert_tokenizer = BertTokenizer.from_pretrained(bert_model)
    t5_tokenizer = T5Tokenizer.from_pretrained(t5_model)
    print(f"Tokenizers loaded")

    model = EncoderDecoder(bert_hidden_size, t5_hidden_size,
                           max_input_length, max_output_length,
                           bert_model, t5_model, batch_size).to(device)
    print(f"Model loaded")
#     print(f"{model.config.decoder_start_token_id}")

    optimizer = optim.Adam(model.parameters(), lr=lr)
    print(f"Otimizer - Adam")

    criterion = nn.CrossEntropyLoss(ignore_index=t5_tokenizer.pad_token_id)
    print(f"CrossEntropyLoss initialized")

    # initialize array of losses
    losses = {'train': {}, "val": {}}

    # for epoch in range(num_epochs):
    with trange(num_epochs) as tr:
        for epoch in tr:

            # Train the model
            model.train()

            batch_loss = 0

            for idx, batch in enumerate(train_dataloder):

                batch_inputs = [data[0] for data in batch]
                batch_sqls = [data[1] for data in batch]

                if DEBUG_FLAG:
                    if epoch == 0 and idx == 0:
                        print(f"batch_inputs - {type(batch_inputs)} {len(batch_inputs)}")
                        print(f"batch_sqls - {type(batch_sqls)} {len(batch_sqls)}")

#                 for temp_i, temp in enumerate(batch_inputs):
#                     print(f"batch_inputs - {batch_inputs[temp_i]}")
#                     print(f"batch_sqls - {batch_sqls[temp_i]}")

                tokenized_inputs = bert_tokenizer(batch_inputs,
                                                  add_special_tokens=True,
                                                  padding="max_length", #True,
                                                  max_length=max_input_length,
                                                  #pad_to_max_length=True,
                                                  return_tensors='pt',
                                                  truncation=True)

                encoder_input_ids = tokenized_inputs["input_ids"].to(device)
                encoder_input_attention_mask = tokenized_inputs["attention_mask"].to(device)

#                 print(f"encoder_input_ids - {encoder_input_ids}")
                tokenized_outputs = t5_tokenizer(batch_sqls,
                                                 add_special_tokens=True,
                                                 padding="max_length", #True,
                                                 max_length=max_output_length,
                                                 #pad_to_max_length=True,
                                                 return_tensors='pt',
                                                 truncation=True)


                decoder_input_ids = tokenized_outputs["input_ids"].to(device)
                # replace padding token id's of the labels by -100 so it's ignored by the loss
                decoder_input_ids[decoder_input_ids == t5_tokenizer.pad_token_id] = -100
                decoder_attention_mask = tokenized_outputs["attention_mask"].to(device)
#                 labels = None #tokenized_outputs["attention_mask"].to(device)

#                 print(f"decoder_input_ids - {decoder_input_ids}")

                if DEBUG_FLAG and epoch == 0 and idx == 0:
                    print(f"encoder_input_ids - {encoder_input_ids.size()}")
                    print(f"encoder_input_attention_mask - {encoder_input_attention_mask.size()}")
                    print(f"decoder_input_ids - {decoder_input_ids.size()}")
                    print(f"decoder_attention_mask - {decoder_attention_mask.size()}")

                # Clear gradients
                optimizer.zero_grad()

                # # graph
                # g  = create_graph(encoder_input_ids)

                model_output = model(encoder_input_ids,
                               encoder_input_attention_mask,
                               decoder_input_ids,
                               decoder_attention_mask)
#                                labels=labels)

                output = model_output["logits"]
#                 print(f"output - {output.size()}")
#                 print(f"decoder_input_ids - {decoder_input_ids.size()}")

                output_resize = output.view(output.shape[0]*output.shape[1], output.shape[2])
                decoder_input_ids_resize = decoder_input_ids.view(decoder_input_ids.shape[0]*decoder_input_ids.shape[1])

#                 print(f"output_resize - {output_resize.size()}")
#                 print(f"decoder_input_ids_resize - {decoder_input_ids_resize.size()}")

#                 loss = criterion(output_resize, decoder_input_ids_resize)
#                 batch_loss += loss.item()

#                 print(f"output - {model_output}")
                loss = model_output["loss"]
                batch_loss += loss

#                 predicted_classes = torch.argmax(output_resize, dim=-1)

#                 print(f"output_resize - {predicted_classes.size} - {predicted_classes}")
#                 print(f"decoder_input_ids_resize - {decoder_input_ids_resize}")

                # backpropagation
                loss.backward()
                optimizer.step()

                break

            batch_loss /= len(train_dataloder)
            losses['train'][epoch] = f"{batch_loss:.3f}"
            #progress bar
            tr.set_postfix({"epoch_num":epoch,
                            "loss":f"{batch_loss:.10f}"})

            with open(os.path.join(models_directory, "loss.json"), 'w') as f:
                json.dump(losses, f)

            writer.add_scalar('Training loss', batch_loss, global_step=epoch+1)
            # save models
            # if (epoch > 3 and epoch % 5 == 0):
            torch.save(model, os.path.join(models_directory, f"model_{epoch}"))
    torch.save(model, os.path.join(models_directory, f"model_last_{epoch}"))
    print(f"saved: {models_directory} model_last_{epoch}")


In [9]:
# Define hyperparameters

train(train_filepath = "../data/resdsql_pre/preprocessed_dataset_train.json",
      batch_size = 2, #32
      bert_hidden_size = 768,
      t5_hidden_size = 512,
      lr = 1e-4,
      num_epochs = 10, #300
      max_input_length = 43,
      max_output_length = 127,
      bert_model = 'bert-base-uncased',
      t5_model = 't5-small')

Number of batches - 3152
Tokenizers loaded


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded
Otimizer - Adam
CrossEntropyLoss initialized


  0%|          | 0/10 [00:00<?, ?it/s]

batch_inputs - <class 'list'> 2
batch_sqls - <class 'list'> 2
encoder_input_ids - torch.Size([2, 43])
encoder_input_attention_mask - torch.Size([2, 43])
decoder_input_ids - torch.Size([2, 127])
decoder_attention_mask - torch.Size([2, 127])


  0%|          | 0/10 [00:00<?, ?it/s]

bert_output - torch.Size([2, 768])


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
# Model class must be defined somewhere
model_path = os.path.join(os.getcwd(), ".\\models\\BERT_T5_lr0.0001_bs2_bert-base-uncased_t5-small\\model_last_29")

model2 = torch.load(model_path, map_location=torch.device('cpu'))
model2.eval()

In [ ]:
import tqdm

dev_filepath = "../data/resdsql_pre/preprocessed_dataset_test.json"
batch_size = 1
max_input_length = 43
bert_model = 'bert-base-uncased'
t5_model = 't5-small'

In [ ]:
# import time
# start_time = time.time()

# initialize tokenizer
bert_tokenizer = BertTokenizer.from_pretrained(bert_model)
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model)

# if isinstance(tokenizer, T5TokenizerFast):
#     tokenizer.add_tokens([AddedToken(" <="), AddedToken(" <")])

dev_dataset = Text2SQLDataset(
            dir_ = dev_filepath,
            mode = "train")

dev_dataloder = DataLoader(
        dev_dataset,
        batch_size = batch_size,
        shuffle = False,
        collate_fn = lambda x: x,
        drop_last = False
    )

# initialize model

model2.eval()
predict_sqls = []
# for batch in tqdm(dev_dataloder):
for idx, batch in enumerate(dev_dataloder):
    batch_inputs = [data[0] for data in batch]
    batch_db_ids = [data[1] for data in batch]
    batch_tc_original = [data[2] for data in batch]

    tokenized_inputs = bert_tokenizer(batch_inputs,
                                      add_special_tokens=True,
                                      padding="max_length", #True,
                                      max_length=max_input_length,
                                      #pad_to_max_length=True,
                                      return_tensors='pt',
                                      truncation=True)

    encoder_input_ids = tokenized_inputs["input_ids"].to(device)
    encoder_input_attention_mask = tokenized_inputs["attention_mask"].to(device)

    print(f"encoder_input_ids - {encoder_input_ids.size()}")
    print(f"encoder_input_attention_mask - {encoder_input_attention_mask.size()}")

    with torch.no_grad():
        model_outputs = model2.predict(encoder_input_ids, encoder_input_attention_mask,
                                       batch_size=1, t5_tokenizer=t5_tokenizer)

#         model_outputs = model_outputs.view(len(batch_inputs), opt.num_return_sequences, model_outputs.shape[1])

#         predict_sqls += decode_sqls(
#                                     opt.db_path,
#                                     model_outputs,
#                                     batch_db_ids,
#                                     batch_inputs,
#                                     tokenizer,
#                                     batch_tc_original
#                                     )
    break


# new_dir = "/".join(opt.output.split("/")[:-1]).strip()
# if new_dir != "":
#     os.makedirs(new_dir, exist_ok = True)

# # save results
# with open(opt.output, "w", encoding = 'utf-8') as f:
#     for pred in predict_sqls:
#         f.write(pred + "\n")

# end_time = time.time()
# print("Text-to-SQL inference spends {}s.".format(end_time-start_time))

# if opt.mode == "eval":
#     # initialize evaluator
#     evaluator = EvaluateTool()
#     evaluator.register_golds(opt.original_dev_filepath, opt.db_path)
#     spider_metric_result = evaluator.evaluate(predict_sqls)
#     print('exact_match score: {}'.format(spider_metric_result["exact_match"]))
#     print('exec score: {}'.format(spider_metric_result["exec"]))

#     return spider_metric_result["exact_match"], spider_metric_result["exec"]

In [ ]:
model_outputs